In [ ]:
import pandas as pd 
import numpy as np 
import re
import gc

from gensim.models import KeyedVectors
from keras import models
from keras import layers
from keras.optimizers import Adam
from keras.preprocessing import text
from keras.callbacks import EarlyStopping

word_vectors = KeyedVectors.load_word2vec_format("../input/googles-trained-word2vec-model-in-python/GoogleNews-vectors-negative300.bin",binary=True)

In [ ]:
#import data 
train_data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
len(max(train_data['excerpt']))

In [ ]:
#Vectorization　word use google word2vec model 
def Vectorization(data):
    #minumam docment size is 669 word.
    doc_array = np.empty((len(data),991, 300))
    for i in range(len(data)):
        word_array = np.empty((991, 300))
        word_list = re.sub('\W',' ', data[i]).split()
        for r in range(991):
            try:
                word_array[r] = word_vectors[word_list[r]]
            #if word is not exist in word2vec return zero array
            except:
                word_array[r] = 0
        doc_array[i] = word_array    
    Vectorization_data = doc_array.reshape((len(doc_array), 991, 300))
    return Vectorization_data

train_x = Vectorization(train_data['excerpt'])
test_x = Vectorization(test_data['excerpt'])

In [ ]:
#delete word_vectors due to lack of memory
del word_vectors
gc.collect()

In [ ]:
# make target data
target = np.array(train_data['target'])
target = target.reshape(2834, 1)

In [ ]:
#make model 
model = models.Sequential()


model.add(layers.Conv1D(64,5, activation='relu',input_shape =(992, 300, )))
model.add(layers.Conv1D(128,5, activation='relu'))
model.add(layers.Conv1D(256,5, activation='relu'))
model.add(layers.Conv1D(512,5, activation='relu'))
model.add(layers.AveragePooling1D(5))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

adam = Adam(lr=1E-5)
model.compile(loss='mean_squared_error', 
              optimizer=adam
             )

model.summary()

In [ ]:
es_cb = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')
history = model.fit(train_x,
                   target,
                   callbacks=[es_cb],
                   epochs=20,
                   batch_size=16,
                   validation_split=0.2)

In [ ]:
del train_x 
gc.collect()

In [ ]:
predict = model.predict(test_x)

In [ ]:
#make submite csv
submission = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
submission['target'] = predict
submission.to_csv('./submission.csv',index=False)